In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/dialnd/imbalanced-algorithms.git

Cloning into 'imbalanced-algorithms'...
remote: Enumerating objects: 428, done.
remote: Total 428 (delta 0), reused 0 (delta 0), pack-reused 428
Receiving objects: 100% (428/428), 28.65 MiB | 12.60 MiB/s, done.
Resolving deltas: 100% (211/211), done.


# Preparation

In [ ]:
import pandas as pd, numpy as np
from plotly import express as px
#  pip install -U kaleido

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/CMG - Crystal Prediction Project/Ternary Materials Point Group Prediction/Data/NOMAD_2/Classification_Data_8.pkl").dropna()
print(df.shape)
df.head()

(1381099, 24)


,Atomic Number 1,Atomic Number 2,Atomic Number 3,Element_1,Element_2,Element_3,Coefficient 1,Coefficient 2,Coefficient 3,formula_reduced,bravais_lattice,crystal_system,space_group_number,point_group,lattice_parameters,Oxidation 1,Oxidation 2,Oxidation 3,IonicRadius_1,IonicRadius_2,IonicRadius_3,IonizationPot1st_1,IonizationPot1st_2,IonizationPot1st_3
0,3,5,41,Li,B,Nb,4,1,1,BLi4Nb,cF,cubic,216,-43m,"{'a': 6.71860936e-10, 'b': 6.71860936e-10, 'c'...",1,-1,-3,0.755440,1.048943,1.842691,5.3917,8.298,6.7589
1,4,5,41,Be,B,Nb,4,1,1,BBe4Nb,cF,cubic,216,-43m,"{'a': 6.119642980000002e-10, 'b': 6.1196429800...",1,-1,-3,0.643658,1.048943,1.842691,9.3226,8.298,6.7589
2,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,cF,cubic,216,-43m,"{'a': 5.458027e-10, 'b': 5.458027e-10, 'c': 5....",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589
3,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,cF,cubic,225,m-3m,"{'a': 5.420156379999999e-10, 'b': 5.4201563799...",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589
4,4,5,41,Be,B,Nb,2,1,1,BBe2Nb,mS,monoclinic,12,2/m,"{'a': 4.660283620000524e-10, 'b': 8.41781306e-...",2,-1,-3,0.452080,1.048943,1.842691,9.3226,8.298,6.7589


Configure

In [ ]:
FEATURE_NAMES = [
#     "Atomic Number 1","Atomic Number 2","Atomic Number 3",
    "Coefficient 1", "Coefficient 2", "Coefficient 3", 
    "IonizationPot1st_1", "IonizationPot1st_2", "IonizationPot1st_3", 
    "Oxidation 1", "Oxidation 2", "Oxidation 3", # Used to be electronegativity 
    "IonicRadius_1", "IonicRadius_2", "IonicRadius_3"
]

GROUPERS = [
    "Atomic Number 1","Atomic Number 2","Atomic Number 3",
    "Coefficient 1", "Coefficient 2", "Coefficient 3"
]

# Y_NAME = 'crystal_system'
# Y_NAME = 'bravais_lattice'
# Y_NAME = 'crystal_system'
Y_NAME = 'space_group_number'

Filter

In [ ]:
Y_NAME = 'space_group_number'
df[Y_NAME] = df[Y_NAME].astype(int).astype(str)
accepted_classes = (df[Y_NAME].value_counts() > 300).replace(False, np.nan).dropna().index.tolist()
df_filtered = df[df[Y_NAME].isin(accepted_classes)].copy()

print("Previous Data Size:", df.shape[0])
print("New Data Size:", df_filtered.shape[0])
print("Number of Classes Left:", len(accepted_classes))

Previous Data Size: 1381099
New Data Size: 1372271
Number of Classes Left: 50


In [ ]:
df_main = df_filtered[FEATURE_NAMES+[Y_NAME]].drop_duplicates().copy()

In [ ]:
df_main.head()

,Coefficient 1,Coefficient 2,Coefficient 3,IonizationPot1st_1,IonizationPot1st_2,IonizationPot1st_3,Oxidation 1,Oxidation 2,Oxidation 3,IonicRadius_1,IonicRadius_2,IonicRadius_3,space_group_number
0,4,1,1,5.3917,8.298,6.7589,1,-1,-3,0.755440,1.048943,1.842691,216
1,4,1,1,9.3226,8.298,6.7589,1,-1,-3,0.643658,1.048943,1.842691,216
2,2,1,1,9.3226,8.298,6.7589,2,-1,-3,0.452080,1.048943,1.842691,216
3,2,1,1,9.3226,8.298,6.7589,2,-1,-3,0.452080,1.048943,1.842691,225
4,2,1,1,9.3226,8.298,6.7589,2,-1,-3,0.452080,1.048943,1.842691,12


In [ ]:
df_grouped = df_main.groupby(FEATURE_NAMES)[Y_NAME].apply(list).reset_index()

# SMOTE+RFC (Base-Line)

SMOTE

In [ ]:
y = df_grouped[Y_NAME]
X = df_grouped[FEATURE_NAMES]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train.values)
X_train_sc = pd.DataFrame(scaler.transform(X_train.values), columns = X_train.columns, index=X_train.index)
X_test_sc =  pd.DataFrame(scaler.transform(X_test.values), columns = X_test.columns, index=X_test.index)

df_train = X_train_sc.merge(y_train, left_index=True, right_index=True).copy()
df_test = X_test_sc.merge(y_test, left_index=True, right_index=True).copy()

In [ ]:
# y_counts = df_train[Y_NAME].value_counts()
# os_strategy = dict([
#                     (y, 5*size) if size < 15000 else (y, size) 
#                     for (y, size) in zip(y_counts.index, y_counts.values)
#                     ])

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
ml_binner = MultiLabelBinarizer()
ml_binner.fit(df_train[Y_NAME])


y_train =  ml_binner.transform(df_train[Y_NAME])
X_train = df_train[FEATURE_NAMES]

y_test = ml_binner.transform(df_test[Y_NAME])
X_test = df_test[FEATURE_NAMES]

In [ ]:
# import pickle 

# # pickle.dump(X_train_os, open("util/X_train_os.pkl", 'wb'))
# # pickle.dump(y_train_os, open("util/y_train_os.pkl", 'wb'))
# # pickle.dump(X_test_f, open("util/X_test_f.pkl", 'wb'))
# # pickle.dump(y_test_f, open("util/y_test_f.pkl", 'wb'))
# # pickle.dump(ml_binner, open("util/ml_binner.pkl", 'wb'))

In [ ]:
# X_train_os = pickle.load(open("util/X_train_os.pkl", 'rb'))
# y_train_os = pickle.load(open("util/y_train_os.pkl", 'rb'))
# X_test_f = pickle.load(open("util/X_test_f.pkl", 'rb'))
# y_test_f = pickle.load(open("util/y_test_f.pkl", 'rb'))
# ml_binner = pickle.load(open("util/ml_binner.pkl", 'rb'))

# Training

In [ ]:
# !pip install -U scikit-learn==0.21.3

In [ ]:
from imbalanced_alg.smote import SMOTEBoost
from sklearn.ensemble import RandomForestClassifier

In [ ]:
print(np.unique(y_train[:, 7], return_counts=True))
smote_boost = RandomForestClassifier(n_estimators=100, random_state=0)
# smote_boost.fit(X=X_train, y=y_train)
smote_boost.fit(X=X_train, y=y_train[:, 7])


(array([0, 1]), array([256136,    374]))


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
print(np.unique(y_test[:, 7], return_counts=True))
y_pred = smote_boost.predict(X_test)
print(np.unique(y_pred, return_counts=True))


(array([0, 1]), array([64026,   102]))
(array([0, 1]), array([64079,    49]))


In [ ]:
from sklearn.metrics import classification_report
target_names = ml_binner.classes_
print(classification_report(y_test[:, 7], y_pred, target_names=['0','1']))

# print(classification_report(y_test, y_pred, target_names=[str(i) for i in target_names]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     64026
           1       0.92      0.44      0.60       102

    accuracy                           1.00     64128
   macro avg       0.96      0.72      0.80     64128
weighted avg       1.00      1.00      1.00     64128



              precision    recall  f1-score   support

           1       0.62      0.22      0.32       223
          10       0.94      0.93      0.93     12553
         107       1.00      0.69      0.82      4688
          11       0.68      0.22      0.33       114
         119       0.94      0.92      0.93     12325
          12       0.91      0.90      0.91     12619
         123       1.00      0.95      0.98     13785
         127       0.86      0.55      0.67       102
         129       0.81      0.30      0.43       162
         139       0.88      0.86      0.87     11762
          14       0.66      0.34      0.45       427
         140       0.88      0.44      0.59        79
         141       0.75      0.43      0.54       110
         148       0.71      0.67      0.69       274
          15       0.76      0.39      0.52       290
         156       0.98      0.66      0.79      4550
         160       0.37      0.09      0.15      3823
         164       0.81    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.

